In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import IPython
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16,8)

In [ ]:
dat = np.fromfile("/home/faux/a.sr2048000.cu8", dtype=np.uint8).astype(np.float32, casting='safe').view(np.complex64)
SR = 2048000

In [ ]:
def spec(dat, sr=SR):
    plt.specgram(dat, NFFT=32, noverlap=16, Fs=sr)

def freq(dat):
    plt.psd(dat, NFFT=1024, Fs=SR)

def tone(freq, samples):
    return np.exp(-2j*np.pi*-freq/SR*np.arange(samples))

In [ ]:
spec(dat[:SR])

In [ ]:
cut = dat[int(0.6*SR):int(0.95*SR)];
spec(cut)

In [ ]:
shift = cut*np.exp(-2j*np.pi*200e3/SR*np.arange(len(cut)))
spec(shift)

In [ ]:
n_taps = 32
f_bw = 25000
lpf = signal.remez(n_taps, [0, f_bw, f_bw+(SR/2-f_bw)/4, SR/2], [1,0], Hz=SR)
dec = signal.decimate(shift[int(0.02*SR):int(0.098*SR)], 16, zero_phase=True)
spec(dec)

In [ ]:
import scipy.fftpack as ffts
import operator

from itertools import islice

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n // 2):
        yield l[i:i + n]

def best(slic):
    mags = [abs(x*x) for x in ffts.ifft(slic)]
    mags = mags[len(slic)//2:] + mags[:len(slic)//2]
    return max(enumerate(mags), key=operator.itemgetter(1))[0]

_ = plt.plot([16-best(x) for x in chunks(dec, 16)])

In [ ]:
import scipy.fftpack as ffts
import operator

from itertools import islice

def chunks(l, n):
    for i in range(0, len(l), n // 4):
        yield l[i:i + n]

def best(slic):
    mags = (abs(x) for x in ffts.ifft(slic))
    low = sum(islice(mags, len(slic) // 2))
    hi = sum(mags)
    return low - hi

dat = [best(x) for x in chunks(dec, 16)]
_ = plt.plot(dat[600:])


In [ ]:
POW = 100
''.join('1' if x > POW else '0' if x < -POW else '_' for x in dat)

In [ ]:
SYM = 13

def deco(val):
    cur = False
    lo = 0
    hi = 0
    un = 0
    for item in val:
        if item < -POW:
            if cur:
                for i in range(round((hi+un)/SYM)):
                    yield cur
                #yield (lo, hi, un)
                lo = 0; hi = 0; un = 0
                cur = False
            else:
                lo += 1
        elif item > POW:
            if not cur:
                for i in range(round((lo+un)/SYM)):
                    yield cur
                #yield (lo, hi, un)
                lo = 0; hi = 0; un = 0
                cur = True
            else:
                hi += 1
        else:
            un += 1

man = list(deco(dat))
_ = plt.stem(man[56:])

In [ ]:
def deman(arr):
    for i in range(0, len(arr), 2):
        pair = arr[i:i+2]
        if [True, False] == pair:
            yield '1'
        elif [False, True] == pair:
            yield '0'
        else:
            return

bits = ''.join(deman(man[55:]))
bits

In [ ]:
def slide(x):
    for i in range(0, len(x), 8):
        yield x[i:i+8]

[''.join(chr(int(x, 2)) for x in slide(bits[s:])) for s in range(12)]